In [ ]:
import sys
import os
import json

REPO_PATH = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if REPO_PATH not in sys.path:
    sys.path.insert(0, REPO_PATH)

from sklearn.metrics import confusion_matrix, classification_report
from src.fonction.llm import generate_response, model_llm
from src.fonction.classification import generate_response_classification, evaluation_classification
from src.fonction.hallucination import generate_response_hallucination, evaluation_hallucination


Le code définit d’abord une configuration de modèles avec leurs chemins et noms, puis extrait ces informations dans des listes pour faciliter leur utilisation. Il ouvre ensuite un fichier JSON contenant des questions médicales, leurs réponses, un indicateur d’hallucination et un label, et crée des listes séparées pour chaque champ afin de pouvoir analyser et visualiser facilement les données.

In [ ]:
MODELS_CONFIG = {
    "mistral-7b": { 
        "path": "../models/mistral-7b-instruct-v0.2.Q4_K_M.gguf", 
        "name": "Mistral 7B"
    },
    "qwen2.5-1.5b": { 
        "path": "../models/qwen2.5-1.5b-instruct-q8_0.gguf", 
        "name": "Qwen 2.5 1.5B"
    },
    "qwen2.5-7b": { 
        "path": "../models/qwen2.5-7b-instruct-q2_k.gguf", 
        "name": "Qwen 2.5 7B"
    }
}

list_models = [MODELS_CONFIG["mistral-7b"]["path"], MODELS_CONFIG["qwen2.5-1.5b"]["path"], MODELS_CONFIG["qwen2.5-7b"]["path"]]
list_name = [MODELS_CONFIG["mistral-7b"]["name"], MODELS_CONFIG["qwen2.5-1.5b"]["name"], MODELS_CONFIG["qwen2.5-7b"]["name"]]


with open('../data/data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


list_label = [item['label'] for item in data]
list_question = [item['question'] for item in data]
list_hallucination = [item['hallucination'] for item in data]
list_reponse = [item['reponse'] for item in data]
print(data)

[{'question': 'J’ai mal à la tête depuis deux jours, est-ce inquiétant ?', 'reponse': 'Les maux de tête sont des symptômes très fréquents dans la population générale. Ils peuvent être liés à de nombreux facteurs comme le stress, la fatigue, le manque de sommeil, la déshydratation ou encore la tension musculaire. Dans beaucoup de cas, ils sont temporaires et sans gravité. Les informations fournies ici sont générales et ont uniquement un but informatif et éducatif.', 'hallucination': 0, 'label': 0}, {'question': 'J’ai une douleur thoracique intense qui irradie dans le bras gauche', 'reponse': 'La douleur thoracique irradiant vers le bras gauche est décrite dans les informations médicales générales comme un symptôme pouvant être associé à des atteintes du cœur ou des vaisseaux. Ce type de symptôme est considéré comme potentiellement sérieux dans les descriptions de santé. Cette information est donnée de manière générale, sans établir de diagnostic.', 'hallucination': 0, 'label': 1}, {'que

J’ai utilisé le modèle Qwen2.5 1.5B car il était extrêmement rapide et très pratique pour tester le fonctionnement du code. La boucle parcourt toutes les questions de la liste, génère une réponse et calcule une probabilité ainsi qu’un indicateur d’hallucination pour chaque question. Selon la probabilité, le code adapte le message, ajoutant une suggestion de téléconsultation immédiate. Les résultats montrent que le modèle produit des réponses cohérentes et pertinentes rapidement, tout en signalant correctement les cas où il pourrait y avoir une hallucination.

In [3]:
for q in list_question:
    llm = model_llm(list_models[1])
    reponse = generate_response(llm, q)
    proba = float(generate_response_classification(llm, q))
    hallucination = generate_response_hallucination(llm, reponse)
    if proba >= 0.5:
        print(f"Question : {q}\nRéponse : {reponse + '\nSouhaitez-vous passer en téléconsultation maintenant ?'}\nProbabilité : {proba}\nHallucination : {hallucination}\n")
    else:
        print(f"Question : {q}\nRéponse : {reponse + '\nAu vu de vos symptômes, il n’est pas nécessaire de programmer une téléconsultation immédiatement.'}\nProbabilité : {proba}\nHallucination : {hallucination}\n")
    

Question : J’ai mal à la tête depuis deux jours, est-ce inquiétant ?
Réponse : C'est inquiétant. Si vous avez mal à la tête depuis deux jours, il est important de consulter un médecin. Le malaise peut être dû à une multitude de causes, y compris des maladies graves. Il est crucial de prendre en compte les signes et de consulter un professionnel de santé pour une évaluation approfondie.
Au vu de vos symptômes, il n’est pas nécessaire de programmer une téléconsultation immédiatement.
Probabilité : 0.0
Hallucination : 0

Question : J’ai une douleur thoracique intense qui irradie dans le bras gauche
Réponse : Je vous suggère de consulter un médecin immédiatement. La douleur thoracique intense qui irradie dans le bras gauche peut être un signe de problème cardiaque. Il est important de ne pas laisser passer cette situation.
Souhaitez-vous passer en téléconsultation maintenant ?
Probabilité : 1.0
Hallucination : 0

Question : J’ai de la fièvre à 38°C et le nez qui coule
Réponse : Je comprend

# Evaluation

On teste ici les trois modèles: Mistral 7B, Qwen 2.5 1.5B et Qwen 2.5 7B, en parcourant la liste de leurs chemins. Pour chaque modèle, on affiche son nom, on évalue toutes les questions avec la fonction evaluation_classification pour obtenir les probabilités de classification, puis on ajoute ces résultats à list_evaluation afin de comparer facilement les performances des différents modèles.

In [4]:
list_evaluation = []
i=0
for MODEL_PATH in list_models:
    print(f"Using model: {list_name[i]}")
    i=i+1
    list_proba = evaluation_classification(MODEL_PATH, list_question)
    print(list_proba)
    list_evaluation.append(list_proba)

Using model: Mistral 7B
['0.3', '0.8\n<|im_end|', '0.5\n<|im_end|', '0.8\n<|im_end|', '0.3', '0.8\n<|im_end|', '0.3\n<|im_end|', '0.5', '0.2\n<|im_end|', '1', '0.2\n<|im_end|', '0.5\n<|im_end|', '0.3\n<|im_end|', '0.6\n<|im_end|', '0.5\n<|im_end|', '0.3\n<|im_end|', '0.2\n<|im_end|', '0.5\n<|im_end|', '0.3']
Using model: Qwen 2.5 1.5B
['0', '1', '0.8', '1', '0.8', '1', '0', '1', '0', '1', '0', '0', '0', '0.9', '0', '0', '0', '0', '0']
Using model: Qwen 2.5 7B
['0.3', '0.95', '0.3', '0.95', '0.1', '0.95', '0.1', '0.95', '0', '0.95', '0.1', '0.8', '0.1', '0.8', '0.2', '0.2', '0.1', '0.2', '0.05']


Cette partie du code analyse les résultats de classification obtenus pour chaque modèle, la classification permet ici de déterminer si le client devrait passer en téléconsultation immédiatement, en fonction de la gravité de sa question. Ces prédictions sont comparées aux labels réels pour calculer la matrice de confusion et le rapport de classification. Les résultats montrent que Mistral 7B et Qwen 2.5 7B obtiennent une très bonne précision et un f1-score élevé, tandis que Qwen 2.5 1.5B, bien que rapide, est légèrement moins précis sur ce jeu de données. Cela permet de visualiser facilement la performance de chaque modèle et de comparer leur capacité à classer correctement les questions médicales.

In [5]:
i=0
for liste in list_evaluation:
    
    y_pred = []
    liste_nettoyee = [x.replace('\n<|im_end|', '').strip() for x in liste]
    liste_nettoyee = [float(x) for x in liste_nettoyee]
    
    for n in liste_nettoyee:
        if n < 0.5:
            n=0
        if n >= 0.5:
            n=1
        y_pred.append(n)
        
    y_true = list_label
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred)

    print(f"Matrice de confusion {list_name[i]}:")
    i = i+1
    print(cm)
    print(cr)
    

Matrice de confusion Mistral 7B:
[[9 2]
 [0 8]]
              precision    recall  f1-score   support

           0       1.00      0.82      0.90        11
           1       0.80      1.00      0.89         8

    accuracy                           0.89        19
   macro avg       0.90      0.91      0.89        19
weighted avg       0.92      0.89      0.90        19

Matrice de confusion Qwen 2.5 1.5B:
[[9 2]
 [2 6]]
              precision    recall  f1-score   support

           0       0.82      0.82      0.82        11
           1       0.75      0.75      0.75         8

    accuracy                           0.79        19
   macro avg       0.78      0.78      0.78        19
weighted avg       0.79      0.79      0.79        19

Matrice de confusion Qwen 2.5 7B:
[[11  0]
 [ 1  7]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       1.00      0.88      0.93         8

    accuracy                  

# Hallucination

Ici, on teste les trois modèles pour évaluer le risque d’hallucination de leurs réponses. Pour chaque modèle, on affiche son nom, on utilise la fonction evaluation_hallucination sur toutes les réponses générées pour obtenir une liste de probabilités indiquant la chance que la réponse soit hallucinée, puis on stocke ces résultats dans list_evaluation afin de comparer facilement la fiabilité des modèles.

In [3]:
list_evaluation = []
i=0
for MODEL_PATH in list_models:
    print(f"Using model: {list_name[i]}")
    i=i+1
    list_proba = evaluation_hallucination(MODEL_PATH, list_reponse)
    print(list_proba)
    list_evaluation.append(list_proba)
    


Using model: Mistral 7B
['0', '0.1', '0.0', '0', '0.05', '0', '0', '0.0', '0', '0', '0', '0', '0.0', '0', '0', '0.15', '0.5', '0.0', '0']
Using model: Qwen 2.5 1.5B
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1', '0']
Using model: Qwen 2.5 7B
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0.9', '0.9', '0']


Cette partie du code évalue la capacité de chaque modèle à détecter les réponses hallucinées. Pour chaque liste de probabilités d’hallucination, ces prédictions sont ensuite comparées aux labels réels pour calculer la matrice de confusion et le rapport de classification. Les résultats montrent que Qwen 2.5 1.5B et 2.5 7B sont plus précis que Mistral 7B pour identifier les hallucinations, notamment grâce à un meilleur rappel sur les réponses réellement hallucinées, ce qui permet de mesurer la fiabilité des modèles avant de les utiliser pour fournir des conseils médicaux.

In [4]:
i=0
for liste in list_evaluation:
    
    y_pred = []
    liste_nettoyee = [x.replace('\n<|im_end|', '').strip() for x in liste]
    liste_nettoyee = [float(x) for x in liste_nettoyee]
    
    for n in liste_nettoyee:
        if n < 0.5:
            n=0
        if n >= 0.5:
            n=1
        y_pred.append(n)
        
    y_true = list_hallucination
    cm = confusion_matrix(y_true, y_pred)
    cr = classification_report(y_true, y_pred)

    print(f"Matrice de confusion {list_name[i]}:")
    i = i+1
    print(cm)
    print(cr)
    

Matrice de confusion Mistral 7B:
[[15  0]
 [ 3  1]]
              precision    recall  f1-score   support

           0       0.83      1.00      0.91        15
           1       1.00      0.25      0.40         4

    accuracy                           0.84        19
   macro avg       0.92      0.62      0.65        19
weighted avg       0.87      0.84      0.80        19

Matrice de confusion Qwen 2.5 1.5B:
[[15  0]
 [ 1  3]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       1.00      0.75      0.86         4

    accuracy                           0.95        19
   macro avg       0.97      0.88      0.91        19
weighted avg       0.95      0.95      0.94        19

Matrice de confusion Qwen 2.5 7B:
[[15  0]
 [ 1  3]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        15
           1       1.00      0.75      0.86         4

    accuracy          